In [1]:
import requests
from bs4 import BeautifulSoup as bs
import datetime as dt
import time
import json
import pymysql

In [2]:
def read_config(p:str) -> dict:
    """
    p: config file 경로
    """
    with open(p, 'r') as f:
        lines = f.readlines()
    config_dict={}

    for l in lines:
        idx = l.index('=')
        k = l[:idx]
        v = l[idx+1:]
        config_dict[k] = v.rstrip()
    
    return config_dict

In [3]:
baseballdb_config = read_config('./baseballdb_config')
conn = pymysql.connect(**baseballdb_config)

In [9]:
def bb_record():
    now = dt.datetime.now()
    ny = now.strftime('%Y')
    nd = now.strftime('%Y%m%d')
    nm = now.strftime('%m')
    x = '4'
    result_lst = []
    for month in range(int(x.zfill(2)), int(nm)+1):
        url=f'https://sports.news.naver.com/kbaseball/schedule/index?date={nd}&month={month}&year={ny}&teamCode='
        response = requests.get(url)
        soup = bs(response.text, 'lxml')
        divs = soup.select('#calendarWrap>div')
        for div in divs:
            trs = div.select('tr')
            for tr in trs:
                tds = tr.select('td')
                if tr == trs[0]:
                    if len(tds) == 3:
                        break
                    elif tr.select('td.add_state'): 
                        win_pit_name = '' ; lose_pit_name = '' ; fin_bat_name = ''
                        win_pit_inn = '' ; lose_pit_inn = '' ; fin_bat_abs = ''
                        win_pit_hit = '' ; lose_pit_hit = '' ; fin_bat_hthr = ''
                        win_pit_bbhp = '' ; lose_pit_bbhp = '' ; fin_bat_bb = ''
                        win_pit_kk = '' ; lose_pit_kk = '' ; fin_bat_kk = ''
                        win_pit_lr = ''; lose_pit_lr = '' ; fin_bat_rbi = ''
                        win_pit_pn = '' ; lose_pit_pn = '' ; fin_bat_run = ''
                        win_pit_era = ''; lose_pit_era = '' ; fin_bat_ba = ''
                        win_pit = (win_pit_name, win_pit_inn, win_pit_hit, win_pit_bbhp, win_pit_kk, win_pit_lr, win_pit_pn, win_pit_era)
                        lose_pit = (lose_pit_name, lose_pit_inn, lose_pit_hit, lose_pit_bbhp, lose_pit_kk, lose_pit_lr, lose_pit_pn, lose_pit_era)
                        fin_bat = (fin_bat_name, fin_bat_abs, fin_bat_hthr, fin_bat_bb, fin_bat_kk, fin_bat_rbi, fin_bat_run, fin_bat_ba)
                    elif tr.find_all('img')[2]['alt'] == "경기결과":
                        if tr.select('strong.td_score')[0].text.split(':')[0] == tr.select('strong.td_score')[0].text.split(':')[1]:
                            win_pit_name = '' ; lose_pit_name = '' ; fin_bat_name = ''
                            win_pit_inn = '' ; lose_pit_inn = '' ; fin_bat_abs = ''
                            win_pit_hit = '' ; lose_pit_hit = '' ; fin_bat_hthr = ''
                            win_pit_bbhp = '' ; lose_pit_bbhp = '' ; fin_bat_bb = ''
                            win_pit_kk = '' ; lose_pit_kk = '' ; fin_bat_kk = ''
                            win_pit_lr = ''; lose_pit_lr = '' ; fin_bat_rbi = ''
                            win_pit_pn = '' ; lose_pit_pn = '' ; fin_bat_run = ''
                            win_pit_era = ''; lose_pit_era = '' ; fin_bat_ba = ''
                            win_pit = (win_pit_name, win_pit_inn, win_pit_hit, win_pit_bbhp, win_pit_kk, win_pit_lr, win_pit_pn, win_pit_era)
                            lose_pit = (lose_pit_name, lose_pit_inn, lose_pit_hit, lose_pit_bbhp, lose_pit_kk, lose_pit_lr, lose_pit_pn, lose_pit_era)
                            fin_bat = (fin_bat_name, fin_bat_abs, fin_bat_hthr, fin_bat_bb, fin_bat_kk, fin_bat_rbi, fin_bat_run, fin_bat_ba)
                        else:
                            raw_record_url = tr.select('a')[0]['href'].replace('game', 'games')
                            record_url = 'https://api-gw.sports.naver.com/schedule'+raw_record_url
                            record_rp = requests.get(record_url)
                            record = json.loads(record_rp.text)
                            pit_record = record['result']['recordData']['pitchersBoxscore']
                            win_pit_info = [pit_info for pit_info in list(pit_record.values())[0] if pit_info['wls'] == '승'] or \
                                            [pit_info for pit_info in list(pit_record.values())[1] if pit_info['wls'] == '승']
                            lose_pit_info = [pit_info for pit_info in list(pit_record.values())[0] if pit_info['wls'] == '패'] or \
                                            [pit_info for pit_info in list(pit_record.values())[1] if pit_info['wls'] == '패']
                            for wp in win_pit_info:
                                win_pit_name = wp['name']
                                win_pit_inn = wp['inn']
                                win_pit_hit = str(wp['hit'])
                                win_pit_bbhp = str(wp['bbhp'])
                                win_pit_kk = str(wp['kk'])
                                win_pit_lr = str(wp['r'])+'('+str(wp['er'])+')'
                                win_pit_pn = str(wp['bf'])
                                win_pit_era = str(wp['era'])
                                win_pit = (win_pit_name, win_pit_inn, win_pit_hit, win_pit_bbhp, win_pit_kk, win_pit_lr, win_pit_pn, win_pit_era)
                            for lp in lose_pit_info:
                                lose_pit_name = lp['name']
                                lose_pit_inn = lp['inn']
                                lose_pit_hit = str(lp['hit'])
                                lose_pit_bbhp = str(lp['bbhp'])
                                lose_pit_kk = str(lp['kk'])
                                lose_pit_lr = str(lp['r'])+'('+str(lp['er'])+')'
                                lose_pit_pn = str(lp['bf'])
                                lose_pit_era = str(lp['era'])
                                lose_pit = (lose_pit_name, lose_pit_inn, lose_pit_hit, lose_pit_bbhp, lose_pit_kk, lose_pit_lr, lose_pit_pn, lose_pit_era)
                            bat_record = record['result']['recordData']['etcRecords']
                            fin_bat_name = bat_record[0]['result'].split('(')[0]
                            bat_score_record = record['result']['recordData']['battersBoxscore']
                            fin_bat_info = [bat_info for bat_info in list(bat_score_record.values())[2] if bat_info['name'] == fin_bat_name] or \
                                            [bat_info for bat_info in list(bat_score_record.values())[3] if bat_info['name'] == fin_bat_name]
                            for fb in fin_bat_info:
                                fin_bat_abs = str(fb['ab'])
                                fin_bat_hthr = str(fb['hit'])+'('+str(fb['hr'])+')'
                                fin_bat_bb = str(fb['bb'])
                                fin_bat_kk = str(fb['kk'])
                                fin_bat_rbi = str(fb['rbi'])
                                fin_bat_run = str(fb['run'])
                                fin_bat_ba = fb['hra']
                                fin_bat = (fin_bat_name, fin_bat_abs, fin_bat_hthr, fin_bat_bb, fin_bat_kk, fin_bat_rbi, fin_bat_run, fin_bat_ba)
                    else:
                        win_pit_name = '' ; lose_pit_name = '' ; fin_bat_name = ''
                        win_pit_inn = '' ; lose_pit_inn = '' ; fin_bat_abs = ''
                        win_pit_hit = '' ; lose_pit_hit = '' ; fin_bat_hthr = ''
                        win_pit_bbhp = '' ; lose_pit_bbhp = '' ; fin_bat_bb = ''
                        win_pit_kk = '' ; lose_pit_kk = '' ; fin_bat_kk = ''
                        win_pit_lr = ''; lose_pit_lr = '' ; fin_bat_rbi = ''
                        win_pit_pn = '' ; lose_pit_pn = '' ; fin_bat_run = ''
                        win_pit_era = ''; lose_pit_era = '' ; fin_bat_ba = ''
                        win_pit = (win_pit_name, win_pit_inn, win_pit_hit, win_pit_bbhp, win_pit_kk, win_pit_lr, win_pit_pn, win_pit_era)
                        lose_pit = (lose_pit_name, lose_pit_inn, lose_pit_hit, lose_pit_bbhp, lose_pit_kk, lose_pit_lr, lose_pit_pn, lose_pit_era)
                        fin_bat = (fin_bat_name, fin_bat_abs, fin_bat_hthr, fin_bat_bb, fin_bat_kk, fin_bat_rbi, fin_bat_run, fin_bat_ba)
                else:
                    if len(tds) == 3:
                        break
                    if tr.select('td.add_state'): 
                        win_pit_name = '' ; lose_pit_name = '' ; fin_bat_name = ''
                        win_pit_inn = '' ; lose_pit_inn = '' ; fin_bat_abs = ''
                        win_pit_hit = '' ; lose_pit_hit = '' ; fin_bat_hthr = ''
                        win_pit_bbhp = '' ; lose_pit_bbhp = '' ; fin_bat_bb = ''
                        win_pit_kk = '' ; lose_pit_kk = '' ; fin_bat_kk = ''
                        win_pit_lr = ''; lose_pit_lr = '' ; fin_bat_rbi = ''
                        win_pit_pn = '' ; lose_pit_pn = '' ; fin_bat_run = ''
                        win_pit_era = ''; lose_pit_era = '' ; fin_bat_ba = ''
                        win_pit = (win_pit_name, win_pit_inn, win_pit_hit, win_pit_bbhp, win_pit_kk, win_pit_lr, win_pit_pn, win_pit_era)
                        lose_pit = (lose_pit_name, lose_pit_inn, lose_pit_hit, lose_pit_bbhp, lose_pit_kk, lose_pit_lr, lose_pit_pn, lose_pit_era)
                        fin_bat = (fin_bat_name, fin_bat_abs, fin_bat_hthr, fin_bat_bb, fin_bat_kk, fin_bat_rbi, fin_bat_run, fin_bat_ba)
                    elif tr.find_all('img')[2]['alt'] == "경기결과":
                        if tr.select('strong.td_score')[0].text.split(':')[0] == tr.select('strong.td_score')[0].text.split(':')[1]:
                            win_pit_name = '' ; lose_pit_name = '' ; fin_bat_name = ''
                            win_pit_inn = '' ; lose_pit_inn = '' ; fin_bat_abs = ''
                            win_pit_hit = '' ; lose_pit_hit = '' ; fin_bat_hthr = ''
                            win_pit_bbhp = '' ; lose_pit_bbhp = '' ; fin_bat_bb = ''
                            win_pit_kk = '' ; lose_pit_kk = '' ; fin_bat_kk = ''
                            win_pit_lr = ''; lose_pit_lr = '' ; fin_bat_rbi = ''
                            win_pit_pn = '' ; lose_pit_pn = '' ; fin_bat_run = ''
                            win_pit_era = ''; lose_pit_era = '' ; fin_bat_ba = ''
                            win_pit = (win_pit_name, win_pit_inn, win_pit_hit, win_pit_bbhp, win_pit_kk, win_pit_lr, win_pit_pn, win_pit_era)
                            lose_pit = (lose_pit_name, lose_pit_inn, lose_pit_hit, lose_pit_bbhp, lose_pit_kk, lose_pit_lr, lose_pit_pn, lose_pit_era)
                            fin_bat = (fin_bat_name, fin_bat_abs, fin_bat_hthr, fin_bat_bb, fin_bat_kk, fin_bat_rbi, fin_bat_run, fin_bat_ba)
                        else:
                            raw_record_url = tr.select('a')[0]['href'].replace('game', 'games')
                            record_url = 'https://api-gw.sports.naver.com/schedule'+raw_record_url
                            record_rp = requests.get(record_url)
                            record = json.loads(record_rp.text)
                            pit_record = record['result']['recordData']['pitchersBoxscore']
                            win_pit_info = [pit_info for pit_info in list(pit_record.values())[0] if pit_info['wls'] == '승'] or \
                                            [pit_info for pit_info in list(pit_record.values())[1] if pit_info['wls'] == '승']
                            lose_pit_info = [pit_info for pit_info in list(pit_record.values())[0] if pit_info['wls'] == '패'] or \
                                            [pit_info for pit_info in list(pit_record.values())[1] if pit_info['wls'] == '패']
                            for wp in win_pit_info:
                                win_pit_name = wp['name']
                                win_pit_inn = wp['inn']
                                win_pit_hit = str(wp['hit'])
                                win_pit_bbhp = str(wp['bbhp'])
                                win_pit_kk = str(wp['kk'])
                                win_pit_lr = str(wp['r'])+'('+str(wp['er'])+')'
                                win_pit_pn = str(wp['bf'])
                                win_pit_era = str(wp['era'])
                                win_pit = (win_pit_name, win_pit_inn, win_pit_hit, win_pit_bbhp, win_pit_kk, win_pit_lr, win_pit_pn, win_pit_era)
                            for lp in lose_pit_info:
                                lose_pit_name = lp['name']
                                lose_pit_inn = lp['inn']
                                lose_pit_hit = str(lp['hit'])
                                lose_pit_bbhp = str(lp['bbhp'])
                                lose_pit_kk = str(lp['kk'])
                                lose_pit_lr = str(lp['r'])+'('+str(lp['er'])+')'
                                lose_pit_pn = str(lp['bf'])
                                lose_pit_era = str(lp['era'])
                                lose_pit = (lose_pit_name, lose_pit_inn, lose_pit_hit, lose_pit_bbhp, lose_pit_kk, lose_pit_lr, lose_pit_pn, lose_pit_era)
                            bat_record = record['result']['recordData']['etcRecords']
                            fin_bat_name = bat_record[0]['result'].split('(')[0]
                            bat_score_record = record['result']['recordData']['battersBoxscore']
                            fin_bat_info = [bat_info for bat_info in list(bat_score_record.values())[2] if bat_info['name'] == fin_bat_name] or \
                                            [bat_info for bat_info in list(bat_score_record.values())[3] if bat_info['name'] == fin_bat_name]
                            for fb in fin_bat_info:
                                fin_bat_abs = str(fb['ab'])
                                fin_bat_hthr = str(fb['hit'])+'('+str(fb['hr'])+')'
                                fin_bat_bb = str(fb['bb'])
                                fin_bat_kk = str(fb['kk'])
                                fin_bat_rbi = str(fb['rbi'])
                                fin_bat_run = str(fb['run'])
                                fin_bat_ba = fb['hra']
                                fin_bat = (fin_bat_name, fin_bat_abs, fin_bat_hthr, fin_bat_bb, fin_bat_kk, fin_bat_rbi, fin_bat_run, fin_bat_ba)
                    else:
                        win_pit_name = '' ; lose_pit_name = '' ; fin_bat_name = ''
                        win_pit_inn = '' ; lose_pit_inn = '' ; fin_bat_abs = ''
                        win_pit_hit = '' ; lose_pit_hit = '' ; fin_bat_hthr = ''
                        win_pit_bbhp = '' ; lose_pit_bbhp = '' ; fin_bat_bb = ''
                        win_pit_kk = '' ; lose_pit_kk = '' ; fin_bat_kk = ''
                        win_pit_lr = ''; lose_pit_lr = '' ; fin_bat_rbi = ''
                        win_pit_pn = '' ; lose_pit_pn = '' ; fin_bat_run = ''
                        win_pit_era = ''; lose_pit_era = '' ; fin_bat_ba = ''
                        win_pit = (win_pit_name, win_pit_inn, win_pit_hit, win_pit_bbhp, win_pit_kk, win_pit_lr, win_pit_pn, win_pit_era)
                        lose_pit = (lose_pit_name, lose_pit_inn, lose_pit_hit, lose_pit_bbhp, lose_pit_kk, lose_pit_lr, lose_pit_pn, lose_pit_era)
                        fin_bat = (fin_bat_name, fin_bat_abs, fin_bat_hthr, fin_bat_bb, fin_bat_kk, fin_bat_rbi, fin_bat_run, fin_bat_ba)
                result_lst.append([win_pit,lose_pit,fin_bat])
    with conn.cursor() as cursor:
        sql = "SELECT `GameID` FROM `TB_game`"
        cursor.execute(sql) 
        result=cursor.fetchall()
    ID_list=list(result)
    ID_list
    wp_ordered_lst=[] ; lp_ordered_lst=[]; fb_ordered_lst=[]
    for i in range(len(result_lst)):
        wp_ordered_lst.append(result_lst[i][0])
        lp_ordered_lst.append(result_lst[i][1])
        fb_ordered_lst.append(result_lst[i][2])
    nwp_ordered_lst=[] ; nlp_ordered_lst=[]; nfb_ordered_lst=[]
    for wo in range(len(wp_ordered_lst)):
        tem_lst_1=[ID_list[wo]+wp_ordered_lst[wo]]
        nwp_ordered_lst.append(tem_lst_1)
    for lo in range(len(lp_ordered_lst)):
        tem_lst_2=[ID_list[lo]+lp_ordered_lst[lo]]
        nlp_ordered_lst.append(tem_lst_2)
    for fo in range(len(fb_ordered_lst)):
        tem_lst_3=[ID_list[fo]+fb_ordered_lst[fo]]
        nfb_ordered_lst.append(tem_lst_3)
    wp_lst = [x for sublist in nwp_ordered_lst for x in sublist]
    lp_lst = [x for sublist in nlp_ordered_lst for x in sublist]
    fb_lst = [x for sublist in nfb_ordered_lst for x in sublist]
    return wp_lst, lp_lst, fb_lst

In [10]:
bbw, bbl, bbf=bb_record()

In [11]:
bbw

[(1, '문성현', '1', '0', '0', '0', '0(0)', '14', '0.00'),
 (2, '김광현', '5', '4', '3', '3', '1(1)', '87', '1.80'),
 (3, '벤자민', '6', '2', '0', '4', '1(0)', '87', '0.00'),
 (4, '최지강', '0 ⅔', '0', '0', '1', '0(0)', '9', '0.00'),
 (5, '페디', '5', '4', '3', '6', '0(0)', '108', '0.00'),
 (6, '김재웅', '1', '0', '0', '3', '0(0)', '12', '0.00'),
 (7, '이의리', '5', '3', '6', '3', '3(1)', '101', '1.80'),
 (8, '함덕주', '2', '0', '0', '4', '0(0)', '29', '0.00'),
 (9, '나균안', '6 ⅔', '5', '2', '4', '0(0)', '100', '0.00'),
 (10, '이승현', '2', '1', '0', '2', '0(0)', '26', '0.00'),
 (11, '이승현', '1', '1', '0', '0', '0(0)', '18', '0.00'),
 (12, '', '', '', '', '', '', '', ''),
 (13, '플럿코', '5', '2', '4', '4', '0(0)', '89', '0.00'),
 (14, '오원석', '7', '2', '2', '6', '1(1)', '94', '1.29'),
 (15, '정철원', '1', '0', '1', '1', '0(0)', '13', '0.00'),
 (16, '', '', '', '', '', '', '', ''),
 (17, '', '', '', '', '', '', '', ''),
 (18, '최원태', '6', '5', '3', '3', '1(1)', '104', '1.50'),
 (19, '', '', '', '', '', '', '', ''),
 (20, '

In [12]:
bbl

[(1, '장시환', '0 ⅔', '3', '1', '0', '1(1)', '22', '13.50'),
 (2, '앤더슨', '6 ⅔', '6', '2', '2', '3(3)', '91', '4.05'),
 (3, '켈리', '5 ⅓', '8', '0', '3', '6(6)', '85', '10.13'),
 (4, '문경찬', '0', '3', '0', '0', '3(3)', '7', '0.00'),
 (5, '뷰캐넌', '5', '8', '0', '5', '4(4)', '99', '7.20'),
 (6, '주현상', '0', '2', '2', '0', '1(1)', '19', '9.00'),
 (7, '맥카티', '3 ⅓', '10', '3', '1', '8(8)', '90', '21.60'),
 (8, '박영현', '0 ⅔', '2', '0', '1', '1(1)', '11', '6.75'),
 (9, '최원준', '7', '5', '1', '2', '2(2)', '96', '2.57'),
 (10, '김진호', '1', '2', '0', '1', '2(2)', '21', '9.00'),
 (11, '한승주', '2', '2', '0', '2', '1(1)', '29', '4.50'),
 (12, '', '', '', '', '', '', '', ''),
 (13, '후라도', '6', '6', '2', '5', '3(3)', '101', '4.50'),
 (14, '박세웅', '4 ⅔', '8', '3', '8', '3(2)', '99', '3.86'),
 (15, '심창민', '0', '0', '1', '0', '1(1)', '6', '4.50'),
 (16, '', '', '', '', '', '', '', ''),
 (17, '', '', '', '', '', '', '', ''),
 (18, '이민호', '5 ⅓', '5', '2', '1', '2(0)', '88', '0.00'),
 (19, '', '', '', '', '', '', '', ''

In [13]:
bbf

[(1, '이형종', '4', '1(0)', '0', '1', '1', '0', '0.250'),
 (2, '박성한', '4', '1(0)', '0', '0', '2', '0', '0.250'),
 (3, '알포드', '4', '4(1)', '0', '0', '3', '1', '1.000'),
 (4, '로하스', '6', '2(1)', '0', '1', '5', '2', '0.333'),
 (5, '박건우', '4', '2(0)', '0', '1', '1', '1', '0.500'),
 (6, '김휘집', '0', '0(0)', '1', '0', '1', '1', '0.000'),
 (7, '최형우', '1', '1(0)', '0', '0', '2', '0', '0.200'),
 (8, '이천웅', '1', '1(0)', '0', '0', '1', '0', '1.000'),
 (9, '한동희', '4', '1(0)', '0', '0', '2', '0', '0.091'),
 (10, '김지찬', '4', '1(0)', '1', '1', '1', '2', '0.375'),
 (11, '피렐라', '4', '1(1)', '0', '2', '2', '1', '0.182'),
 (12, '', '', '', '', '', '', '', ''),
 (13, '김현수', '4', '1(0)', '0', '0', '1', '1', '0.417'),
 (14, '김현수', '4', '1(0)', '0', '0', '1', '1', '0.417'),
 (15, '김인태', '4', '1(0)', '0', '0', '1', '0', '0.333'),
 (16, '', '', '', '', '', '', '', ''),
 (17, '', '', '', '', '', '', '', ''),
 (18, '', '', '', '', '', '', '', ''),
 (19, '', '', '', '', '', '', '', ''),
 (20, '김성욱', '4', '2(1)', '1',

In [7]:
with conn.cursor() as cursor:
    sql="INSERT INTO `TB_winpitcher`(`GameID`, `Name`, `Inning`, `Hit`, `BBHP`, `StrikeOut`, `Lose_Run`, `PitchingNumber`, `Era`) VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s)"
    cursor.executemany(sql, bbw)
conn.commit()

with conn.cursor() as cursor:
    sql="INSERT INTO `TB_losepitcher`(`GameID`, `Name`, `Inning`, `Hit`, `BBHP`, `StrikeOut`, `Lose_Run`, `PitchingNumber`, `Era`) VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s)"
    cursor.executemany(sql, bbl)
conn.commit()

with conn.cursor() as cursor:
    sql="INSERT INTO `TB_batter`(`GameID`, `Name`, `AtBats`, `Hit_Homerun`, `BB`, `StrikeOut`, `RBI`, `Run`, `BA`) VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s)"
    cursor.executemany(sql, bbf)
conn.commit()

In [1]:
from modules.first_baseball_crawler import weekagoCrawler as wc
from modules.update_baseball_crawler import yesterCrawler as yc
from modules.baseball_db import MariaDB
import datetime as dt
from datetime import timedelta
import time

if __name__ == "__main__":
    now = dt.datetime.now()
    baseballdb_config = MariaDB.read_config('./modules/baseballdb_config')
    db=MariaDB(**baseballdb_config)
    def bb_week():
        weekago = now  - timedelta(days=7)
        wka=weekago.strftime('%Y-%m-%d')
        count = db.select_count(wka)
        if count != 0:
            pass
        else:
            cg = wc.crawl_game()
            cg_lst = [tuple(x) for x in cg]
            db.insert_many('TB_game', ['PlayedAt', 'AwayTeam', 'AwayScore', 'HomeTeam', 'HomeScore', 'Stadium', 'Broadcast'], cg_lst)   
            crw, crl, crf=wc.crawl_record()
            db.insert_many('TB_winpitcher', ['GameID', 'Name', 'Inning', 'Hit', 'BBHP', 'StrikeOut', 'Lose_Run', 'PitchingNumber', 'Era'], crw)
            db.insert_many('TB_losepitcher', ['GameID', 'Name', 'Inning', 'Hit', 'BBHP', 'StrikeOut', 'Lose_Run', 'PitchingNumber', 'Era'], crl)
            db.insert_many('TB_batter', ['GameID', 'Name', 'AtBats', 'Hit_Homerun', 'BB', 'StrikeOut', 'RBI', 'Run', 'BA'], crf)
        return

    def bb_yester():
        yester = now  - timedelta(days=1)
        yt=yester.strftime('%Y-%m-%d')
        count = db.select_count(yt)
        if count != 0:
            pass
        else:
            cyg = yc.crawl_ytgame()
            cyg_lst = [tuple(x) for x in cyg]
            db.insert_many('TB_game', ['PlayedAt', 'AwayTeam', 'AwayScore', 'HomeTeam', 'HomeScore', 'Stadium', 'Broadcast'], cyg_lst)
            ncrw, ncrl, ncrf=yc.crawl_ytrecord()
            db.insert_many('TB_winpitcher', ['GameID', 'Name', 'Inning', 'Hit', 'BBHP', 'StrikeOut', 'Lose_Run', 'PitchingNumber', 'Era'], ncrw)
            db.insert_many('TB_losepitcher', ['GameID', 'Name', 'Inning', 'Hit', 'BBHP', 'StrikeOut', 'Lose_Run', 'PitchingNumber', 'Era'], ncrl)
            db.insert_many('TB_batter', ['GameID', 'Name', 'AtBats', 'Hit_Homerun', 'BB', 'StrikeOut', 'RBI', 'Run', 'BA'], ncrf)
        return
               
    while True:
        CurTm = time.localtime()
        if CurTm.tm_hour == 12 and CurTm.tm_min == 10:
            bb_week(); bb_yester()
            print(f'업데이트 시간:{now}')
            break
        time.sleep(60)
        

ModuleNotFoundError: No module named 'modules'

In [2]:
import time
time.().tm_hour

23